In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import stmpy
import numpy as np
import cancelv3 as cancel
from decimal import *
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import pandas as pd
import scipy.fftpack as fftp
import os
from scipy import signal
import scipy
import matplotlib.pyplot as plt
import matplotlib
import time
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [38]:
# create NeuralNetwork class
class NeuralNetwork:

    # intialize variables in class
    def __init__(self, inputs, outputs):
        self.inputs  = inputs
        self.outputs = outputs
        # initialize weights as .50 for simplicity
        self.weights = np.array([[.50], [.50], [.50]])
        self.error_history = []
        self.epoch_list = []

    #activation function ==> S(x) = 1/1+e^(-x)
    def sigmoid(self, x, deriv=False):
        if deriv == True:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    # data will flow through the neural network.
    def feed_forward(self):
        self.hidden = self.sigmoid(np.dot(self.inputs, self.weights))

    # going backwards through the network to update weights
    def backpropagation(self):
        self.error  = self.outputs - self.hidden
        delta = self.error * self.sigmoid(self.hidden, deriv=True)
        self.weights += np.dot(self.inputs.T, delta)

    # train the neural net for 25,000 iterations
    def train(self, epochs=25000):
        for epoch in range(epochs):
            # flow forward and produce an output
            self.feed_forward()
            # go back though the network to make corrections based on the output
            self.backpropagation()    
            # keep track of the error history over each epoch
            self.error_history.append(np.average(np.abs(self.error)))
            self.epoch_list.append(epoch)

    # function to predict output on new and unseen input data                               
    def predict(self, new_input):
        prediction = self.sigmoid(np.dot(new_input, self.weights))
        return prediction

# create neural network   
# NN = NeuralNetwork(inputs, outputs)
# # train neural network
# NN.train()

# # create two new examples to predict                                   
# example = np.array([[1, 1, 0]])
# example_2 = np.array([[0, 1, 1]])

# # print the predictions for both examples                                   
# print(NN.predict(example), ' - Correct: ', example[0][0])
# print(NN.predict(example_2), ' - Correct: ', example_2[0][0])

# # plot the error over the entire training duration
# plt.figure(figsize=(15,5))
# plt.plot(NN.epoch_list, NN.error_history)
# plt.xlabel('Epoch')
# plt.ylabel('Error')
# plt.show()

In [33]:
file_name = "trx_2-"
folder_name = "Albert_Data/2019-02-26/"

# trx_2.npy is a numpy array compiled from a calibration measurement
dat=load(folder_name+"trx_2.npy")
geo=dat[:,0]
z=dat[:,1]

In [34]:
# def __init__(self, xdata, ydata, dt1, zdata=None, optimize=True,
#              p=None,**kwarg):
#     self.dt1 = dt1
#     if p == None:
#         self.p = [len(xdata),0,0]
#     else:
#         self.p = p
#     [self.transf, self.freqs] = self.get_transf(xdata, ydata, self.p, dt1)
#     self.xdata = [xdata]
#     self.ydata = [ydata]
#     self.N = len(self.xdata)

def fit_cancel(array, n=2):
    t=np.linspace(0,len(array),len(array))
    fit = np.polyfit(t,array,n)
    corrective = fit[n]*np.ones(np.shape(t))
    for j in np.arange(n):
        corrective += fit[j]*(t**(n-j))
    return array - corrective

# x is the array, n is the length of each segment, o is the number of overlaping elements adjacent segments share
def chop(x,n,o):
    segs = np.zeros(((len(x)-o)//(n-o),n),dtype=np.complex)
    j = 0
    while len(x) >= n:
#             i = 0
#             mini_seg = np.zeros(n)
#             while i < n:
#                 mini_seg[i] = x[i]
#                 i+=1
#             segs[j,:] = mini_seg
        segs[j,:] = fit_cancel(x[:n])
        j+=1
        x=x[n-o:]
#             x = x[i-o:]
    return segs

# Applies a Kaiser Window paramaterized by b to each segment
def window(segs,b):
    seg_num = len(segs[:,0])
    seg_leng = len(segs[0,:])
    for i in range(0,seg_num):
        segs[i,:] = segs[i,:]*np.kaiser((seg_leng),b)
    return segs

# Fourier Transforms each Row, given time between samples, dt
def fourier(segs,dt):
    seg_num = len(segs[:,0])
    freqs = np.fft.fftfreq(len(segs[0,:]),dt)
    for i in range(0,seg_num):
        segs[i,:] = np.fft.fft(segs[i,:])
    return freqs,segs

def get_transf(xdata, ydata, p, dt):

    n = int(p[0])
    o = int(p[1])
    b = p[2]

    top = chop(ydata,n,o)
    top = window(top,b)
    [freqs, ftop] = fourier(top,dt)

    bottom = chop(xdata,n,o)
    bottom = window(bottom,b)
    [freqs, fbottom] = fourier(bottom,dt)
    
    return ftop,fbottom,freqs

#     num_segs = len(fbottom[:,0])
#     numer = 0
#     denomer = 0
#     for i in range(0,num_segs):
#         numer += ftop[i,:]*np.conjugate(fbottom[i,:])
#         denomer += fbottom[i,:]*np.conjugate(fbottom[i,:])

#     numer = numer/num_segs
#     denomer = denomer/num_segs

#     tf = numer/denomer
#     tf_spl = interp1d(freqs, tf)
#     return tf_spl,freqs

In [35]:
tip_ft, geo_ft, freqs = get_transf(geo[:100000],z[:100000],[20000,12500,5],1/10000)
tip_ft = tip_ft.T
geo_ft = geo_ft.T

In [36]:
print(shape(tip_ft),shape(geo_ft),shape(freqs))

(20000, 11) (20000, 11) (20000,)


In [40]:
# create neural network   
NN = NeuralNetwork(geo_ft[:,0],tip_ft[:,0])
# train neural network
NN.train()

# create two new examples to predict                                   
example = geo_ft[:,1]
example_2 = geo_ft[:,2]

# print the predictions for both examples                                   
print(NN.predict(example), ' - Correct: ', example[0][0])
print(NN.predict(example_2), ' - Correct: ', example_2[0][0])

# plot the error over the entire training duration
plt.figure(figsize=(15,5))
plt.plot(NN.epoch_list, NN.error_history)
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\jgoet\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-6e89f640b9fe>", line 4, in <module>
    NN.train()
  File "<ipython-input-38-20c52e154dff>", line 33, in train
    self.feed_forward()
  File "<ipython-input-38-20c52e154dff>", line 21, in feed_forward
    self.hidden = self.sigmoid(np.dot(self.inputs, self.weights))
ValueError: shapes (20000,) and (3,1) not aligned: 20000 (dim 0) != 3 (dim 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jgoet\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exception, another exc

ValueError: shapes (20000,) and (3,1) not aligned: 20000 (dim 0) != 3 (dim 0)